# Metadata

**L1 Taxonomy** - SQL in Python

**L2 Taxonomy** - Object-Relational Mapping

**Subtopic** - Implementing SQLAlchemy ORM with Flask for Database-Driven Web Applications

**Use Case** - Develop a simple command-line application in Python using SQLAlchemy ORM for creating, reading, updating, and deleting records in a SQLite database file.

**Programming Language** - Python

**Target Model** - GPT-4o

# Setup

```requirements.txt
sqlalchemy==2.0.41
```


# Prompt

## **Problem Overview**

You are required to build a **command-line interface (CLI)** application using **Python and SQLAlchemy ORM** that manages a SQLite database. The application should track people, their contact details, and group memberships. The system must support **CRUD operations**, **auditing**, **many-to-many and one-to-one relationships**, **transactional batch updates**, and **advanced filtering via CLI commands**.

All functionality must be encapsulated within a **single, independent Python module** that cleanly separates responsibilities using functions and classes.


## **Input Format**

Run the application using:

```bash
python main.py <command> [options]
```

### Supported CLI commands:

1. `create-person --name NAME --age AGE`
2. `add-contact --person-id ID --email EMAIL [--phone PHONE]`
3. `create-group --name NAME`
4. `assign-group --person-id ID --group-id ID`
5. `read [--min-age AGE] [--max-age AGE] [--name-contains TEXT] [--group NAME] [--sort-by FIELD] [--limit N] [--offset N]`
6. `update-person --id ID [--name NAME] [--age AGE]`
7. `delete-person --id ID`
8. `batch-update-age --ids ID1 ID2 ... --age AGE`
9. `export --format FORMAT --entity ENTITY --output FILE`
10. `import --format FORMAT --entity ENTITY --input FILE`


## **Output Format**

* On success:

  ```
  Created Person with ID: 1
  Added ContactInfo to Person ID: 1
  Created Group with ID: 2
  Assigned Person ID: 1 to Group ID: 2
  Updated Person ID: 1
  Deleted Person ID: 1
  Exported records to people.json
  ```

* On reading records:

  ```
  ID: 1, Name: Alice, Age: 30
  Groups: Engineers, Developers
  Email: alice@example.com | Phone: 1234567890
  ```

* On error:

  ```
  Error: Invalid email format.
  Error: Person with ID 99 not found.
  Error: Transaction failed. Rolled back.
  ```


## **Examples**

### Example 1:

```bash
python main.py create-person --name Alice --age 30
```

**Output:**

```
Created Person with ID: 1
```

### Example 2:

```bash
python main.py read --min-age 25 --group Developers --sort-by age
```

**Output:**

```
ID: 1, Name: Alice, Age: 30
Groups: Developers
Email: alice@example.com | Phone: 1234567890
```

### Example 3:

```bash
python main.py batch-update-age --ids 1 2 3 --age 40
```

**Output:**

```
Updated age to 40 for Persons: 1, 2, 3
```


# Requirements


## **Explicit Requirements**

* Use SQLAlchemy ORM to define the following tables:

  * `Person`: id, name, age
  * `ContactInfo`: id, email, phone, person_id (One-to-One with `Person`)
  * `Group`: id, name
  * `person_group`: association table (Many-to-Many between `Person` and `Group`)
  * `ChangeLog`: id, timestamp, entity, action (create/update/delete), details

* Commands must support:

  * Creating/updating/deleting `Person`, `Group`, and `ContactInfo`
  * Assigning and listing group memberships
  * Full changelog auditing for all mutations
  * Reading persons with filters:

    * `--min-age`, `--max-age`
    * `--name-contains`, `--group`
    * `--sort-by`, `--limit`, `--offset`
  * Batch age update across multiple people using transactions
  * Importing/exporting records to/from JSON or CSV


## **Implicit Requirements**

* Validate:

  * Name is not empty and <= 100 characters
  * Age > 0
  * Email contains `@`
  * Phone (if provided) is digits only
* When updating contact info, create it if it doesn’t exist
* Show `None` for missing contact info in read output
* Ensure all foreign key constraints are respected
* Implement rollback if any operation in a batch update fails
* All output must be clean and readable


## **Function Signature(s) and/or Class Definition(s)**

You must encapsulate logic into modular components like the following:

```python
class DatabaseManager:
    def __init__(self, db_url: str = "sqlite:///people.db"): ...

    def create_person(self, name: str, age: int) -> int: ...
    def add_or_update_contact(self, person_id: int, email: str, phone: str = None) -> bool: ...
    def create_group(self, name: str) -> int: ...
    def assign_group(self, person_id: int, group_id: int) -> bool: ...
    def read_people(self, filters: dict) -> list[dict]: ...
    def update_person(self, person_id: int, name: str = None, age: int = None) -> bool: ...
    def delete_person(self, person_id: int) -> bool: ...
    def batch_update_age(self, person_ids: list[int], age: int) -> bool: ...
    def export_data(self, entity: str, format: str, file_path: str) -> bool: ...
    def import_data(self, entity: str, format: str, file_path: str) -> bool: ...
```

This class must internally handle sessions, validation, commits, rollbacks, and logging of operations to the `ChangeLog`.


## **Edge Cases**

* Batch update with one invalid ID should rollback the entire operation
* Deleting a person should cascade or delete their contact/group links
* Importing malformed JSON should fail gracefully
* `read` with no filters must return all people
* Duplicate group assignment should not crash the app
* Exporting with unsupported format should print a meaningful error
* Updating a person with no field provided should raise an error


## **Constraints**

* Must work with **Python 3.8+**
* Use only standard Python libraries and **SQLAlchemy ORM**
* No raw SQL is allowed
* Must run as a **single script/module** (no required file separation)
* Must work with SQLite and auto-create `people.db` on first run
* CLI parsing must be done using the `argparse` module
* Code should follow good structure: validation, error handling, transaction control
* Must never crash due to invalid input or system errors


In [ ]:
# code

"""
People Management CLI Application
Manages people, contact details, and group memberships using SQLAlchemy ORM.
"""

import argparse
import json
import csv
import sys
from datetime import datetime
from typing import List, Dict

from sqlalchemy import create_engine, Column, Integer, String, DateTime, ForeignKey, Table
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import sessionmaker, relationship, Session
from sqlalchemy.exc import SQLAlchemyError

Base = declarative_base()

# Association table for many-to-many relationship between Person and Group
person_group = Table(
    'person_group',
    Base.metadata,
    Column('person_id', Integer, ForeignKey('person.id'), primary_key=True),
    Column('group_id', Integer, ForeignKey('group.id'), primary_key=True)
)


class Person(Base):
    """Implementation of class Person"""
    __tablename__ = 'person'

    id = Column(Integer, primary_key=True)
    name = Column(String(100), nullable=False)
    age = Column(Integer, nullable=False)

    # One-to-One relationship with ContactInfo
    contact_info = relationship("ContactInfo", back_populates="person",
                                uselist=False, cascade="all, delete-orphan")

    # Many-to-Many relationship with Group
    groups = relationship("Group", secondary=person_group,
                          back_populates="people")


class ContactInfo(Base):
    """Implementation of class ContactInfo"""
    __tablename__ = 'contact_info'

    id = Column(Integer, primary_key=True)
    email = Column(String(255), nullable=False)
    phone = Column(String(20), nullable=True)
    person_id = Column(Integer, ForeignKey('person.id'),
                       nullable=False, unique=True)

    # One-to-One relationship with Person
    person = relationship("Person", back_populates="contact_info")


class Group(Base):
    """Implementation of class Group"""
    __tablename__ = 'group'

    id = Column(Integer, primary_key=True)
    name = Column(String(100), nullable=False, unique=True)

    # Many-to-Many relationship with Person
    people = relationship("Person", secondary=person_group,
                          back_populates="groups")


class ChangeLog(Base):
    """Implementation of class ChangeLog"""
    __tablename__ = 'change_log'

    id = Column(Integer, primary_key=True)
    timestamp = Column(DateTime, default=datetime.utcnow)
    entity = Column(String(50), nullable=False)
    action = Column(String(20), nullable=False)
    details = Column(String(500), nullable=False)


class DatabaseManager:
    """Implementation of main class DatabaseManager"""
    def __init__(self, db_url: str = "sqlite:///people.db"):
        self.engine = create_engine(db_url)
        Base.metadata.create_all(self.engine)
        self.Session = sessionmaker(bind=self.engine)

    def _log_change(self, session: Session, entity: str, action: str, details: str):
        """Log changes to the ChangeLog table"""
        log_entry = ChangeLog(entity=entity, action=action, details=details)
        session.add(log_entry)

    def _validate_name(self, name: str) -> bool:
        """Validate name is not empty and <= 100 characters"""
        return bool(name and name.strip() and len(name.strip()) <= 100)

    def _validate_age(self, age: int) -> bool:
        """Validate age > 0"""
        return age > 0

    def _validate_email(self, email: str) -> bool:
        """Validate email contains @"""
        return '@' in email

    def _validate_phone(self, phone: str) -> bool:
        """Validate phone is digits only"""
        return phone.isdigit() if phone else True

    def create_person(self, name: str, age: int) -> int:
        """Create a new person and return their ID"""
        if not self._validate_name(name):
            raise ValueError("Name must not be empty and <= 100 characters")
        if not self._validate_age(age):
            raise ValueError("Age must be > 0")

        session = self.Session()
        try:
            person = Person(name=name.strip(), age=age)
            session.add(person)
            session.commit()

            self._log_change(session, "Person", "create", f"Created person: {name}, age: {age}")
            session.commit()

            return person.id
        except SQLAlchemyError as e:
            session.rollback()
            raise RuntimeError(f"Database error: {str(e)}") from e
        finally:
            session.close()

    def add_or_update_contact(self, person_id: int, email: str, phone: str = None) -> bool:
        """Add or update contact info for a person"""
        if not self._validate_email(email):
            raise ValueError("Invalid email format")
        if phone and not self._validate_phone(phone):
            raise ValueError("Phone must contain only digits")

        session = self.Session()
        try:
            person = session.query(Person).filter_by(id=person_id).first()
            if not person:
                raise ValueError(f"Person with ID {person_id} not found")

            if person.contact_info:
                # Update existing contact info
                person.contact_info.email = email
                person.contact_info.phone = phone
                action = "update"
            else:
                # Create new contact info
                contact_info = ContactInfo(email=email, phone=phone, person_id=person_id)
                session.add(contact_info)
                action = "create"

            session.commit()

            self._log_change(session, "ContactInfo", action,
                             f"Contact info for person {person_id}: {email}, {phone}")
            session.commit()

            return True
        except SQLAlchemyError as e:
            session.rollback()
            raise RuntimeError(f"Database error: {str(e)}") from e
        finally:
            session.close()

    def create_group(self, name: str) -> int:
        """Create a new group and return its ID"""
        if not self._validate_name(name):
            raise ValueError("Group name must not be empty and <= 100 characters")

        session = self.Session()
        try:
            group = Group(name=name.strip())
            session.add(group)
            session.commit()

            self._log_change(session, "Group", "create", f"Created group: {name}")
            session.commit()

            return group.id
        except SQLAlchemyError as e:
            session.rollback()
            if "UNIQUE constraint failed" in str(e):
                raise ValueError(f"Group '{name}' already exists")
            raise RuntimeError(f"Database error: {str(e)}")
        finally:
            session.close()

    def assign_group(self, person_id: int, group_id: int) -> bool:
        """Assign a person to a group"""
        session = self.Session()
        try:
            person = session.query(Person).filter_by(id=person_id).first()
            if not person:
                raise ValueError(f"Person with ID {person_id} not found")

            group = session.query(Group).filter_by(id=group_id).first()
            if not group:
                raise ValueError(f"Group with ID {group_id} not found")

            # Check if already assigned
            if group not in person.groups:
                person.groups.append(group)
                session.commit()

                self._log_change(session, "PersonGroup", "create",
                                 f"Assigned person {person_id} to group {group_id}")
                session.commit()

            return True
        except SQLAlchemyError as e:
            session.rollback()
            raise RuntimeError(f"Database error: {str(e)}") from e
        finally:
            session.close()

    def read_people(self, filters: dict) -> List[Dict]:
        """Read people with optional filters"""
        session = self.Session()
        try:
            query = session.query(Person)

            # Apply filters
            if filters.get('min_age'):
                query = query.filter(Person.age >= filters['min_age'])
            if filters.get('max_age'):
                query = query.filter(Person.age <= filters['max_age'])
            if filters.get('name_contains'):
                query = query.filter(Person.name.contains(filters['name_contains']))
            if filters.get('group'):
                query = query.join(Person.groups).filter(Group.name == filters['group'])

            # Apply sorting
            if filters.get('sort_by'):
                if filters['sort_by'] == 'name':
                    query = query.order_by(Person.name)
                elif filters['sort_by'] == 'age':
                    query = query.order_by(Person.age)
                elif filters['sort_by'] == 'id':
                    query = query.order_by(Person.id)

            # Apply pagination
            if filters.get('offset'):
                query = query.offset(filters['offset'])
            if filters.get('limit'):
                query = query.limit(filters['limit'])

            people = query.all()

            result = []
            for person in people:
                groups = [group.name for group in person.groups]
                contact_info = {
                    'email': person.contact_info.email if person.contact_info else None,
                    'phone': person.contact_info.phone if person.contact_info else None
                }

                result.append({
                    'id': person.id,
                    'name': person.name,
                    'age': person.age,
                    'groups': groups,
                    'contact_info': contact_info
                })

            return result
        except SQLAlchemyError as e:
            raise RuntimeError(f"Database error: {str(e)}") from e
        finally:
            session.close()

    def update_person(self, person_id: int, name: str = None, age: int = None) -> bool:
        """Update a person's information"""
        if name is None and age is None:
            raise ValueError("At least one field (name or age) must be provided for update")

        if name is not None and not self._validate_name(name):
            raise ValueError("Name must not be empty and <= 100 characters")
        if age is not None and not self._validate_age(age):
            raise ValueError("Age must be > 0")

        session = self.Session()
        try:
            person = session.query(Person).filter_by(id=person_id).first()
            if not person:
                raise ValueError(f"Person with ID {person_id} not found")

            changes = []
            if name is not None:
                person.name = name.strip()
                changes.append(f"name: {name}")
            if age is not None:
                person.age = age
                changes.append(f"age: {age}")

            session.commit()

            self._log_change(session, "Person", "update",
                             f"Updated person {person_id}: {', '.join(changes)}")
            session.commit()

            return True
        except SQLAlchemyError as e:
            session.rollback()
            raise RuntimeError(f"Database error: {str(e)}") from e
        finally:
            session.close()

    def delete_person(self, person_id: int) -> bool:
        """Delete a person and their associated data"""
        session = self.Session()
        try:
            person = session.query(Person).filter_by(id=person_id).first()
            if not person:
                raise ValueError(f"Person with ID {person_id} not found")

            person_name = person.name
            session.delete(person)
            session.commit()

            self._log_change(session, "Person", "delete",
                             f"Deleted person {person_id}: {person_name}")
            session.commit()

            return True
        except SQLAlchemyError as e:
            session.rollback()
            raise RuntimeError(f"Database error: {str(e)}") from e
        finally:
            session.close()

    def batch_update_age(self, person_ids: List[int], age: int) -> bool:
        """Update age for multiple people in a single transaction"""
        if not self._validate_age(age):
            raise ValueError("Age must be > 0")

        session = self.Session()
        try:
            # Validate all person IDs exist
            people = session.query(Person).filter(Person.id.in_(person_ids)).all()
            found_ids = {person.id for person in people}
            missing_ids = set(person_ids) - found_ids

            if missing_ids:
                raise ValueError(f"Person(s) with ID(s) {sorted(missing_ids)} not found")

            # Update all people
            for person in people:
                person.age = age

            session.commit()

            self._log_change(session, "Person", "update",
                             f"Batch updated age to {age} for persons: {sorted(person_ids)}")
            session.commit()

            return True
        except SQLAlchemyError as e:
            session.rollback()
            raise RuntimeError(f"Database error: {str(e)}") from e
        finally:
            session.close()

    def export_data(self, entity: str, export_format: str, file_path: str) -> bool:
        """Export data to JSON or CSV format"""
        if export_format not in ['json', 'csv']:
            raise ValueError("Format must be 'json' or 'csv'")
        if entity not in ['person', 'group', 'contact']:
            raise ValueError("Entity must be 'person', 'group', or 'contact'")

        session = self.Session()
        try:
            data = []

            if entity == 'person':
                people = session.query(Person).all()
                for person in people:
                    groups = [group.name for group in person.groups]
                    contact_info = person.contact_info
                    data.append({
                        'id': person.id,
                        'name': person.name,
                        'age': person.age,
                        'groups': groups,
                        'email': contact_info.email if contact_info else None,
                        'phone': contact_info.phone if contact_info else None
                    })

            elif entity == 'group':
                groups = session.query(Group).all()
                for group in groups:
                    people = [person.name for person in group.people]
                    data.append({
                        'id': group.id,
                        'name': group.name,
                        'people': people
                    })

            elif entity == 'contact':
                contacts = session.query(ContactInfo).all()
                for contact in contacts:
                    data.append({
                        'id': contact.id,
                        'email': contact.email,
                        'phone': contact.phone,
                        'person_id': contact.person_id,
                        'person_name': contact.person.name
                    })

            # Write to file
            if export_format == 'json':
                with open(file_path, 'w', encoding='utf-8') as f:
                    json.dump(data, f, indent=2)
            elif export_format == 'csv':
                if data:
                    with open(file_path, 'w', newline='', encoding='utf-8') as f:
                        writer = csv.DictWriter(f, fieldnames=data[0].keys())
                        writer.writeheader()
                        writer.writerows(data)

            return True
        except Exception as e:
            raise RuntimeError(f"Export failed: {str(e)}") from e
        finally:
            session.close()

    def import_data(self, entity: str, input_format: str, file_path: str) -> bool:
        """Import data from JSON or CSV format"""
        if input_format not in ['json', 'csv']:
            raise ValueError("Format must be 'json' or 'csv'")
        if entity not in ['person', 'group']:
            raise ValueError("Entity must be 'person' or 'group'")

        try:
            # Read data from file
            if input_format == 'json':
                with open(file_path, 'r', encoding='utf-8') as f:
                    data = json.load(f)
            elif input_format == 'csv':
                with open(file_path, 'r', encoding='utf-8') as f:
                    reader = csv.DictReader(f)
                    data = list(reader)

            session = self.Session()
            try:
                if entity == 'person':
                    for item in data:
                        person = Person(
                            name=item['name'],
                            age=int(item['age'])
                        )
                        session.add(person)
                        session.flush()  # Get the ID

                        if item.get('email'):
                            contact_info = ContactInfo(
                                email=item['email'],
                                phone=item.get('phone'),
                                person_id=person.id
                            )
                            session.add(contact_info)

                elif entity == 'group':
                    for item in data:
                        group = Group(name=item['name'])
                        session.add(group)

                session.commit()
                return True
            except SQLAlchemyError as e:
                session.rollback()
                raise RuntimeError(f"Import failed: {str(e)}") from e
            finally:
                session.close()

        except Exception as e:
            raise RuntimeError(f"Import failed: {str(e)}") from e


def main():
    parser = argparse.ArgumentParser(description="People Management CLI")
    subparsers = parser.add_subparsers(dest='command', help='Available commands')

    # Create person
    create_person_parser = subparsers.add_parser('create-person', help='Create a new person')
    create_person_parser.add_argument('--name', required=True, help='Person name')
    create_person_parser.add_argument('--age', type=int, required=True, help='Person age')

    # Add contact
    add_contact_parser = subparsers.add_parser('add-contact', help='Add contact info to a person')
    add_contact_parser.add_argument('--person-id', type=int, required=True, help='Person ID')
    add_contact_parser.add_argument('--email', required=True, help='Email address')
    add_contact_parser.add_argument('--phone', help='Phone number')

    # Create group
    create_group_parser = subparsers.add_parser('create-group', help='Create a new group')
    create_group_parser.add_argument('--name', required=True, help='Group name')

    # Assign group
    assign_group_parser = subparsers.add_parser('assign-group', help='Assign person to group')
    assign_group_parser.add_argument('--person-id', type=int, required=True, help='Person ID')
    assign_group_parser.add_argument('--group-id', type=int, required=True, help='Group ID')

    # Read people
    read_parser = subparsers.add_parser('read', help='Read people with optional filters')
    read_parser.add_argument('--min-age', type=int, help='Minimum age filter')
    read_parser.add_argument('--max-age', type=int, help='Maximum age filter')
    read_parser.add_argument('--name-contains', help='Name contains filter')
    read_parser.add_argument('--group', help='Group name filter')
    read_parser.add_argument('--sort-by', choices=['name', 'age', 'id'], help='Sort by field')
    read_parser.add_argument('--limit', type=int, help='Limit results')
    read_parser.add_argument('--offset', type=int, help='Offset results')

    # Update person
    update_person_parser = subparsers.add_parser('update-person', help='Update person information')
    update_person_parser.add_argument('--id', type=int, required=True, help='Person ID')
    update_person_parser.add_argument('--name', help='New name')
    update_person_parser.add_argument('--age', type=int, help='New age')

    # Delete person
    delete_person_parser = subparsers.add_parser('delete-person', help='Delete a person')
    delete_person_parser.add_argument('--id', type=int, required=True, help='Person ID')

    # Batch update age
    batch_update_parser = subparsers.add_parser('batch-update-age',
                                                help='Batch update age for multiple people')
    batch_update_parser.add_argument('--ids', type=int, nargs='+', required=True, help='Person IDs')
    batch_update_parser.add_argument('--age', type=int, required=True, help='New age')

    # Export
    export_parser = subparsers.add_parser('export', help='Export data')
    export_parser.add_argument('--format', choices=['json', 'csv'],
                               required=True, help='Export format')
    export_parser.add_argument('--entity', choices=['person', 'group', 'contact'],
                               required=True, help='Entity to export')
    export_parser.add_argument('--output', required=True, help='Output file path')

    # Import
    import_parser = subparsers.add_parser('import', help='Import data')
    import_parser.add_argument('--format', choices=['json', 'csv'],
                               required=True, help='Import format')
    import_parser.add_argument('--entity', choices=['person', 'group'],
                               required=True, help='Entity to import')
    import_parser.add_argument('--input', required=True, help='Input file path')

    args = parser.parse_args()

    if not args.command:
        parser.print_help()
        return

    db = DatabaseManager()

    try:
        if args.command == 'create-person':
            person_id = db.create_person(args.name, args.age)
            print(f"Created Person with ID: {person_id}")

        elif args.command == 'add-contact':
            db.add_or_update_contact(args.person_id, args.email, args.phone)
            print(f"Added ContactInfo to Person ID: {args.person_id}")

        elif args.command == 'create-group':
            group_id = db.create_group(args.name)
            print(f"Created Group with ID: {group_id}")

        elif args.command == 'assign-group':
            db.assign_group(args.person_id, args.group_id)
            print(f"Assigned Person ID: {args.person_id} to Group ID: {args.group_id}")

        elif args.command == 'read':
            filters = {}
            if args.min_age:
                filters['min_age'] = args.min_age
            if args.max_age:
                filters['max_age'] = args.max_age
            if args.name_contains:
                filters['name_contains'] = args.name_contains
            if args.group:
                filters['group'] = args.group
            if args.sort_by:
                filters['sort_by'] = args.sort_by
            if args.limit:
                filters['limit'] = args.limit
            if args.offset:
                filters['offset'] = args.offset

            people = db.read_people(filters)
            for person in people:
                print(f"ID: {person['id']}, Name: {person['name']}, Age: {person['age']}")
                if person['groups']:
                    print(f"Groups: {', '.join(person['groups'])}")

                email = person['contact_info']['email'] or 'None'
                phone = person['contact_info']['phone'] or 'None'
                print(f"Email: {email} | Phone: {phone}")
                print()

        elif args.command == 'update-person':
            db.update_person(args.id, args.name, args.age)
            print(f"Updated Person ID: {args.id}")

        elif args.command == 'delete-person':
            db.delete_person(args.id)
            print(f"Deleted Person ID: {args.id}")

        elif args.command == 'batch-update-age':
            db.batch_update_age(args.ids, args.age)
            print(f"Updated age to {args.age} for Persons: {', '.join(map(str, args.ids))}")

        elif args.command == 'export':
            db.export_data(args.entity, args.format, args.output)
            print(f"Exported records to {args.output}")

        elif args.command == 'import':
            db.import_data(args.entity, args.format, args.input)
            print(f"Imported records from {args.input}")

    except (ValueError, RuntimeError) as e:
        print(f"Error: {str(e)}")
        sys.exit(1)
    except Exception as e:
        print(f"Error: {str(e)}")
        sys.exit(1)


if __name__ == '__main__':
    main()


In [ ]:
# tests

"""Module for testing the DatabaseManager class."""

import unittest
import os
import tempfile
import json
from main import DatabaseManager


class TestDatabaseManager(unittest.TestCase):
    """Test suite for DatabaseManager SQLAlchemy CLI application."""

    def setUp(self):
        """Set up test environment."""
        self.test_db_fd, self.test_db_path = tempfile.mkstemp(suffix=".db")
        os.close(self.test_db_fd)
        self.db_url = f"sqlite:///{self.test_db_path}"

        self.db_manager = DatabaseManager(self.db_url)
        self.temp_json_fd, self.temp_json_path = tempfile.mkstemp(
            suffix=".json"
        )
        self.temp_csv_fd, self.temp_csv_path = tempfile.mkstemp(suffix=".csv")
        os.close(self.temp_json_fd)
        os.close(self.temp_csv_fd)

    def tearDown(self):
        """Tear down test environment."""
        if hasattr(self.db_manager, "session"):
            self.db_manager.session.close()
        if hasattr(self.db_manager, "engine"):
            self.db_manager.engine.dispose()
        for file_path in [
            self.test_db_path, self.temp_json_path, self.temp_csv_path
        ]:
            if os.path.exists(file_path):
                os.unlink(file_path)

    def test_create_person_returns_valid_id(self):
        """Test create_person returns a valid integer ID."""
        person_id = self.db_manager.create_person("Alice Johnson", 30)
        self.assertIsInstance(person_id, int)
        self.assertGreater(person_id, 0)

    def test_create_person_invalid_name_raises_error(self):
        """Test create_person raises error for empty name validation."""
        with self.assertRaises(ValueError):
            self.db_manager.create_person("", 30)

    def test_create_person_invalid_age_raises_error(self):
        """Test create_person raises error for age <= 0 validation."""
        with self.assertRaises(ValueError):
            self.db_manager.create_person("John Doe", 0)

    def test_add_contact_info_success(self):
        """Test add_or_update_contact returns True for valid input."""
        person_id = self.db_manager.create_person("Bob Smith", 25)
        result = self.db_manager.add_or_update_contact(
            person_id, "bob@example.com", "1234567890"
        )
        self.assertTrue(result)

    def test_add_contact_invalid_email_raises_error(self):
        """Test add_or_update_contact raises error for invalid email."""
        person_id = self.db_manager.create_person("Test Person", 30)
        with self.assertRaises(ValueError):
            self.db_manager.add_or_update_contact(
                person_id, "invalid-email", "1234567890"
            )

    def test_create_group_returns_valid_id(self):
        """Test create_group returns a valid integer ID."""
        group_id = self.db_manager.create_group("Engineers")
        self.assertIsInstance(group_id, int)
        self.assertGreater(group_id, 0)

    def test_assign_person_to_group_success(self):
        """Test assign_group returns True for valid assignment."""
        person_id = self.db_manager.create_person("Charlie Brown", 35)
        group_id = self.db_manager.create_group("Developers")
        result = self.db_manager.assign_group(person_id, group_id)
        self.assertTrue(result)

    def test_read_people_with_age_filter(self):
        """Test read_people filters by min_age correctly."""
        self.db_manager.create_person("Alice", 30)
        self.db_manager.create_person("Bob", 25)
        people = self.db_manager.read_people({"min_age": 28})
        self.assertEqual(len(people), 1)

    def test_read_people_with_name_filter(self):
        """Test read_people filters by name_contains correctly."""
        self.db_manager.create_person("Alice Johnson", 30)
        self.db_manager.create_person("Bob Smith", 25)
        people = self.db_manager.read_people({"name_contains": "Alice"})
        self.assertEqual(len(people), 1)

    def test_update_person_success(self):
        """Test update_person returns True for valid update."""
        person_id = self.db_manager.create_person("David Wilson", 28)
        result = self.db_manager.update_person(
            person_id, name="David Wilson Jr."
        )
        self.assertTrue(result)

    def test_update_nonexistent_person_raises_error(self):
        """Test update_person raises error for non-existent ID."""
        with self.assertRaises(ValueError):
            self.db_manager.update_person(999, name="Nobody")

    def test_delete_person_success(self):
        """Test delete_person returns True for valid deletion."""
        person_id = self.db_manager.create_person("Eva Martinez", 32)
        result = self.db_manager.delete_person(person_id)
        self.assertTrue(result)

    def test_batch_update_valid_ids_success(self):
        """Test batch_update_age returns True for all valid IDs."""
        person1_id = self.db_manager.create_person("Frank", 45)
        person2_id = self.db_manager.create_person("Grace", 29)
        result = self.db_manager.batch_update_age([person1_id, person2_id], 40)
        self.assertTrue(result)

    def test_batch_update_invalid_id_rollback(self):
        """Test batch_update_age raises error and rolls back on invalid ID."""
        person1_id = self.db_manager.create_person("Henry", 33)
        with self.assertRaises(Exception):
            self.db_manager.batch_update_age([person1_id, 999], 40)

    def test_export_persons_to_json_success(self):
        """Test export_data returns True for valid JSON export."""
        self.db_manager.create_person("John Doe", 30)
        result = self.db_manager.export_data(
            "person", "json", self.temp_json_path
        )
        self.assertTrue(result)

    def test_import_persons_from_json_success(self):
        """Test import_data returns True for valid JSON import."""
        test_data = [{"name": "Imported Person", "age": 35}]
        with open(self.temp_json_path, "w") as f:
            json.dump(test_data, f)
        result = self.db_manager.import_data(
            "person", "json", self.temp_json_path
        )
        self.assertTrue(result)

    def test_import_invalid_json_raises_error(self):
        """Test import_data raises error for malformed JSON."""
        with open(self.temp_json_path, "w") as f:
            f.write("invalid json")
        with self.assertRaises(Exception):
            self.db_manager.import_data("person", "json", self.temp_json_path)

    def test_read_people_pagination_limit(self):
        """Test read_people respects limit parameter for pagination."""
        self.db_manager.create_person("Person1", 30)
        self.db_manager.create_person("Person2", 25)
        self.db_manager.create_person("Person3", 35)
        people = self.db_manager.read_people({"limit": 2})
        self.assertEqual(len(people), 2)


if __name__ == "__main__":
    unittest.main()


# Model Breaking Proof

#### Model Breaking Task URL: https://rlhf-v3.turing.com/prompt/018ec112-6b81-4676-b3d7-f75bdb1c3155?origin=https%3A%2F%2Flabeling-ms.turing.com&redirect_url=https%3A%2F%2Flabeling-ms.turing.com%2Fconversations%2F74412%2Fview

#### Model code:

```python
import sys
import argparse
import json
import csv
from datetime import datetime
from sqlalchemy import create_engine, Column, Integer, String, ForeignKey, Table, DateTime
from sqlalchemy.ext.declarative import declarative_base
from sqlalchemy.orm import relationship, sessionmaker, backref

Base = declarative_base()

# Association table between Person and Group
person_group = Table(
    'person_group', Base.metadata,
    Column('person_id', Integer, ForeignKey('persons.id')),
    Column('group_id', Integer, ForeignKey('groups.id'))
)

class Person(Base):
    __tablename__ = 'persons'
    id = Column(Integer, primary_key=True)
    name = Column(String(100), nullable=False)
    age = Column(Integer, nullable=False)
    contact_info = relationship("ContactInfo", uselist=False, back_populates="person")
    groups = relationship(
        "Group",
        secondary=person_group,
        back_populates="members"
    )

class ContactInfo(Base):
    __tablename__ = 'contact_info'
    id = Column(Integer, primary_key=True)
    email = Column(String, nullable=False)
    phone = Column(String)
    person_id = Column(Integer, ForeignKey('persons.id'))
    person = relationship("Person", back_populates="contact_info")

class Group(Base):
    __tablename__ = 'groups'
    id = Column(Integer, primary_key=True)
    name = Column(String, nullable=False, unique=True)
    members = relationship(
        "Person",
        secondary=person_group,
        back_populates="groups"
    )

class ChangeLog(Base):
    __tablename__ = 'changelog'
    id = Column(Integer, primary_key=True)
    timestamp = Column(DateTime, default=datetime.utcnow)
    entity = Column(String, nullable=False)
    action = Column(String, nullable=False)
    details = Column(String, nullable=False)

class DatabaseManager: # <Issue>: create_group, assign_group, read_people_with_filters, update_person, delete_person, batch_update_age, export_data, import_data these are the methods that are not implemented. Incomplete solution.
    def __init__(self, db_url: str = "sqlite:///people.db"):
        self.engine = create_engine(db_url)
        Base.metadata.create_all(self.engine)
        self.Session = sessionmaker(bind=self.engine)

    def create_person(self, name: str, age: int) -> int:
        if not name or len(name) > 100:
            raise ValueError("Name is required and must be <= 100 characters")
        if age <= 0:
            raise ValueError("Age must be a positive integer")

        session = self.Session()
        try:
            person = Person(name=name, age=age)
            session.add(person)
            session.commit()
            self.log_change(session, 'Person', 'create', f'Created Person ID: {person.id}')
            print(f"Created Person with ID: {person.id}")
            return person.id
        except Exception as e:
            session.rollback()
            print(f"Error: {str(e)}")
        finally:
            session.close()

    def log_change(self, session, entity, action, details):
        log_entry = ChangeLog(entity=entity, action=action, details=details)
        session.add(log_entry)

    # Implement other methods following similar patterns: validation, session handling, logging

    def add_or_update_contact(self, person_id: int, email: str, phone: str = None) -> bool:
        if '@' not in email:
            raise ValueError("Invalid email format")
        if phone and not phone.isdigit():
            raise ValueError("Phone must be digits only")

        session = self.Session()
        try:
            person = session.query(Person).filter(Person.id == person_id).first()
            if not person:
                raise ValueError(f"Person with ID {person_id} not found")

            contact = person.contact_info
            if not contact:
                contact = ContactInfo(email=email, phone=phone, person=person)
                session.add(contact)
            else:
                contact.email = email
                contact.phone = phone

            session.commit()
            self.log_change(session, 'ContactInfo', 'update' if contact else 'create',
                            f'Updated ContactInfo for Person ID: {person_id}') # <Issue>: Log written after commit; this won’t persist unless followed by a second commit. Also, logging for update, delete, and assign_group is not implemented.

            print(f"Added ContactInfo to Person ID: {person_id}")
            return True
        except Exception as e:
            session.rollback()
            print(f"Error: {str(e)}")
        finally:
            session.close()

    # Other CRUD methods...


def setup_argparse():
    # <Issue>: Incomplete CLI Commands, only 'create-person' is implemented.
    parser = argparse.ArgumentParser(description='Manage a simple person-contact system.')
    subparsers = parser.add_subparsers(dest='command')

    # Define the create-person command
    parser_create_person = subparsers.add_parser('create-person')
    parser_create_person.add_argument('--name', required=True, help='Name of the person')
    parser_create_person.add_argument('--age', required=True, type=int, help='Age of the person')

    # Define other commands similarly...

    return parser

def main():
    parser = setup_argparse()
    args = parser.parse_args()

    db_manager = DatabaseManager()

    if args.command == 'create-person':
        db_manager.create_person(args.name, args.age)

    # Handle other command executions...

if __name__ == "__main__":
    main()

# <Issue>: Transactional batch logic not handled in the solution.
```